
 Objectif global du TP

    Construire un système expert médical capable de diagnostiquer une maladie à partir de symptômes, en s’appuyant sur des données provenant de plusieurs sources (JSON, PostgreSQL, API FDA), après nettoyage, consolidation, fouille de motifs fréquents, évaluation d’algorithmes et déploiement sous forme d’API Flask, avec intégration continue des nouvelles données.

 ÉTAPE 1 : GÉNÉRATION ET COLLECTE DE DONNÉES
Objectif :

Créer un jeu de données réaliste à partir :

    D’un fichier JSON (patients générés),

    D’une base PostgreSQL (consultations),

    D’une API publique (openFDA).

Justification technique :

    Dans un contexte médical, il est essentiel d’avoir des données hétérogènes pour améliorer la robustesse du système. Comme les vraies données médicales sont rarement accessibles librement, nous simulons des patients réalistes avec Faker et croisons plusieurs sources pour plus de diversité et de fiabilité.



In [ ]:
# patients_generator.py
import json
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker('fr_FR')

diagnosisSymptomsMap = {
    "grippe": ["fièvre", "toux", "fatigue", "courbatures"],
    "COVID-19": ["fièvre", "toux sèche", "perte d’odorat", "fatigue"],
    "migraine": ["maux de tête", "nausées", "sensibilité à la lumière"],
    "angine": ["gorge irritée", "fièvre", "douleur en avalant"],
    "varicelle": ["éruption cutanée", "fièvre", "démangeaisons"],
    "asthme": ["essoufflement", "sifflements", "toux"],
    "allergie": ["éternuements", "yeux rouges", "démangeaisons"],
    "gastro-entérite": ["nausées", "vomissements", "diarrhée", "crampes"],
    "hypertension": ["maux de tête", "vertiges", "saignement de nez"]
}
def generatePatient(i):
    diagnosis = random.choice(list(diagnosisSymptomsMap.keys()))
    available_symptoms = diagnosisSymptomsMap[diagnosis]
    k = random.randint(2, min(4, len(available_symptoms)))
    symptoms = random.sample(available_symptoms, k=k)

    return {
        "patient_id": f"P{str(i).zfill(3)}",
        "age": random.randint(18, 90),
        "gender": random.choice(["M", "F"]),
        "symptoms": symptoms,
        "diagnosis": diagnosis,
        "timestamp": (datetime.now() - timedelta(days=random.randint(0, 365))).isoformat(),
        "notes": fake.sentence() if random.random() > 0.3 else None
    }


patients = [generatePatient(i) for i in range(1, 51)]
with open("patients.json", "w", encoding="utf-8") as f:
    json.dump(patients, f, indent=2, ensure_ascii=False)
print("✅ patients.json généré.")


✅ patients.json généré.


In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.1 MB/s eta 0:00:00


In [25]:
import sqlite3
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker('fr_FR')

diagnosisSymptomsMap = {
    "grippe": ["fièvre", "toux", "fatigue", "courbatures"],
    "COVID-19": ["fièvre", "toux sèche", "perte d’odorat", "fatigue"],
    "migraine": ["maux de tête", "nausées", "sensibilité à la lumière"],
    "angine": ["gorge irritée", "fièvre", "douleur en avalant"],
    "asthme": ["essoufflement", "sifflements", "toux"],
    "allergie": ["éternuements", "yeux rouges", "démangeaisons"],
    "gastro-entérite": ["nausées", "vomissements", "diarrhée", "crampes"],
    "hypertension": ["maux de tête", "vertiges", "saignement de nez"],
    "varicelle": ["fièvre", "éruption cutanée", "démangeaisons"]
}

conn = sqlite3.connect("medical.db")
cursor = conn.cursor()

# Création de la table si elle n'existe pas
cursor.execute("""
CREATE TABLE IF NOT EXISTS consultations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    patientCode TEXT,
    consultationDate TEXT,
    symptoms TEXT,
    diagnosis TEXT,
    doctorName TEXT,
    medicalCenter TEXT,
    status TEXT
)
""")

statuses = ["terminée", "en attente", "en cours"]
centers = ["CHU Lyon", "Hôpital Saint-Louis", "Clinique Pasteur"]

for _ in range(50):
    diagnosis = random.choice(list(diagnosisSymptomsMap.keys()))
    availableSymptoms = diagnosisSymptomsMap[diagnosis]
    k = random.randint(2, min(4, len(availableSymptoms)))
    symptoms = random.sample(availableSymptoms, k=k)

    cursor.execute("""
        INSERT INTO consultations
        (patientCode, consultationDate, symptoms, diagnosis, doctorName, medicalCenter, status)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        f"P{random.randint(1, 999):03}",
        (datetime.now() - timedelta(days=random.randint(0, 365))).isoformat(),
        ', '.join(symptoms),
        diagnosis,
        f"Dr {fake.last_name()}",
        random.choice(centers),
        random.choice(statuses)
    ))

conn.commit()
cursor.close()
conn.close()
print("✅ Données insérées dans SQLite.")


✅ Données insérées dans SQLite.


In [26]:
import requests
import pandas as pd

url = "https://api.fda.gov/drug/label.json"
params = {"limit": 50}  # On prend 10 entrées pour commencer

response = requests.get(url, params=params)
data = response.json()

# Normaliser le JSON pour extraire les colonnes utiles
df = pd.json_normalize(data["results"])

# Exemple : extrait nom médicament + usage
meds_df = df[["id", "openfda.brand_name", "purpose", "indications_and_usage"]]

print(meds_df.head())



                                     id openfda.brand_name  \
0  ca7bbcc8-2354-375c-e053-2995a90a72a0          [SILICEA]   
1  f229e866-5775-4e42-a316-8480dd92fec6                NaN   
2  b8f5797b-73e1-4201-b824-2cdd50c90497         [Betadine]   
3  01f4b0f6-94df-fd91-e063-6394a90a7997                NaN   
4  f8ce57b8-ebf7-4dc1-96ec-c8fbc41c17ff                NaN   

                                             purpose  \
0  [USES USES: Temporary Relief - Acne, Boils* * ...   
1                                [Purpose Sunscreen]   
2                     [Purpose First aid Antiseptic]   
3         [USES To relieve the symptoms of itching.]   
4                                                NaN   

                               indications_and_usage  
0  [INDICATIONS Condition listed above or as dire...  
1  [Uses Multi-purpose mineral powder provides br...  
2  [Uses First aid to help prevent infection in m...  
3        [INDICATIONS Indications: MEZEREUM Itching]  
4  [INDICATIONS

In [28]:
# 🧠 INTENTION GENERALE
# Construire un système expert médical qui prend des symptômes en entrée
# et retourne une maladie probable, en se basant sur des données réelles (PostgreSQL, JSON, API).

# 1. CHARGEMENT ET NETTOYAGE DES DONNÉES -------------------------------------------------

import pandas as pd
import json
import psycopg2
import requests

# Chargement JSON
with open("patients.json", "r", encoding="utf-8") as f:
    patients = json.load(f)
patients_df = pd.json_normalize(patients)
patients_df["symptoms"] = patients_df["symptoms"].apply(lambda x: [s.strip().lower() for s in x])

import sqlite3

conn = sqlite3.connect("medical.db")
consultations_df = pd.read_sql("SELECT * FROM consultations;", conn)
conn.close()


consultations_df["symptoms"] = consultations_df["symptoms"].apply(lambda x: [s.strip().lower() for s in x])

# Chargement API (openFDA)
url = "https://api.fda.gov/drug/label.json"
params = {"limit": 50}
api_response = requests.get(url, params=params)
api_data = api_response.json()
api_df = pd.json_normalize(api_data["results"])
api_df["brand_name"] = api_df["openfda.brand_name"].apply(lambda x: x[0] if isinstance(x, list) else None)
api_df["purpose"] = api_df["purpose"].apply(lambda x: x[0] if isinstance(x, list) else None)
api_df["indications"] = api_df["indications_and_usage"].apply(lambda x: x[0] if isinstance(x, list) else None)
api_df = api_df[["brand_name", "purpose", "indications"]].dropna()